# This notebook is to be run before EDA_Main
This notebook gets data from the Capital Bikeshare system logs, and saves a csv-formatted file in a format suitale for consumptions for our analysis.

In [1]:
import pandas as pd

In [2]:
filenames=["2012-Q1-Trips-History-Data.csv",
          "2012-Q2-Trips-History-Data.csv",
           "2012-Q3-Trips-History-Data.csv",
           "2012-Q4-Trips-History-Data.csv",
           
           "2013-Q1-Trips-History-Data.csv",
           "2013-Q2-Trips-History-Data.csv",
           "2013-Q3-Trips-History-Data.csv",
           "2013-Q4-Trips-History-Data2.csv"]

path="./"

# File Header exploration:

All files have 7 columns. The columns are not stored in the same order across the files. For instance, "End Station" can be found as the 2nd or 4th column, depending on the file. 

In the final table, these are the columns we expect to have:

1. Trip Duration 

Column names in file: "Duration", "Total duration (ms)", or "Duration (ms)".
Data Format:

2. Start Date

Column names in file: always "Start Date". 
Data Format:

3. End Date

Column names in file: always "End Date".
Data Format:

4. Start Station

Column names in file: "Start station" or "Start Station" (case changes).
Data Format:

5. End Station

Column names in file: "End station" or "End Station" (case changes).
Data Format:

6. Bike Number

Column names in file: "Bike Number" or "Bike#". 
Data Format:

7. Member Type

Column names in file: "Type", "Bike Key", "Subscriber Type", "Subscription Type", and "Member Type.
Data Format:

Let's read in the data and join all the files into one single dataframe. 


In [3]:
#This is a dictionary to help us figure out which column is which
colNames={}
colNames["duration"]=["duration","total duration (ms)","duration (ms)"]
colNames["startDate"]=["start date","start time"]
colNames["endDate"]=["end date","end time"]
colNames["startStation"]=["start station"]
colNames["endStation"]=["end station"]
colNames["bikeNum"]=["bike number","bike#"]
colNames["memberType"]=["type", "bike key", "subscriber type",
                        "subscription type","member type"]

In [4]:
#master is the FINAL table, which will eventually go to the tsv file
master = pd.DataFrame()
#masterDict has temporary data
masterDict=dict()
for i,iFile in enumerate(filenames):
    
    print "Reading:"+iFile
    
    #read the file as a dataframe
    curFile=pd.read_csv(path+iFile)
    
    #get columns in this file
    colsInFile=list(curFile)
    
    #convert all column names to lowercase
    colsInFile=[x.lower() for x in colsInFile]
    curFile.columns = map(str.lower, curFile.columns)
    
    #1. for each column in our master table, find the "equivalent" column
    #in the current file (b/c the column name could be difference but equivalent)
    #2. get the values for the equivalent column
    #3. and if the column exists in the master table, just append the data. 
    for key, valuess in colNames.iteritems():
        for value in valuess:
            for col in colsInFile:
                if value==col:
                    if key not in list(master.columns) and i==0:
                        masterDict[key]=curFile[col].values.tolist()
                    else:
                        [masterDict[key].append(n) for n in curFile[col].values.tolist()]
print "Creating dataframe"
master=pd.DataFrame(masterDict)
print "Formatting date column startDate"#format dates to datetime objects
master["startDate"]=pd.to_datetime(master["startDate"])
print "Formatting date column endDate"
master["endDate"]=pd.to_datetime(master["endDate"])

#Get the time the bike was ridden for...
master['tmp']=master['endDate']-master['startDate']

#...but we want it in minutes...
theListOfMinutes=[]
numEntries=len(master)
for i in range(0,numEntries):
    if not i%500000:
        print i,".",numEntries
    theDuration=master['tmp'][i]
    theListOfMinutes.append(theDuration.seconds/60)
del master['tmp']
master['durationMins']=theListOfMinutes

#save to a file...takes a while (over 7M entries using provided files)
print "Saving to file..."
master.to_csv('master.tsv',engine='C',sep='\t')
print 'Done'

Reading:2012-Q1-Trips-History-Data.csv
Reading:2012-Q2-Trips-History-Data.csv
Reading:2012-Q3-Trips-History-Data.csv
Reading:2012-Q4-Trips-History-Data.csv
Reading:2013-Q1-Trips-History-Data.csv
Reading:2013-Q2-Trips-History-Data.csv
Reading:2013-Q3-Trips-History-Data.csv
Reading:2013-Q4-Trips-History-Data2.csv
Creating dataframe
Formatting date column startDate
Formatting date column endDate
0 . 4631685
500000 . 4631685
1000000 . 4631685
1500000 . 4631685
2000000 . 4631685
2500000 . 4631685
3000000 . 4631685
3500000 . 4631685
4000000 . 4631685
4500000 . 4631685
Saving to file...
Done
